In [13]:
import os

import aesara
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import scipy
import scipy.stats as stats

from aesara.tensor.nnet.basic import softmax
from scipy.interpolate import BSpline

## 1. Load data, split into train and test sets

In [2]:
try:
    wells = pd.read_csv(os.path.join("..", "data", "wells.csv"))
except:
    wells = pd.read_csv(pm.get_data("wells.csv"))

In [3]:
np.random.seed(1)
train_id = wells.sample(n=2000).index
test_id = wells.index[~wells.index.isin(train_id)]
y_train = wells.loc[train_id, "switch"].to_numpy()
y_test = wells.loc[test_id, "switch"].to_numpy()

## 2. Feature engineering

In [4]:
def bs(x, knots, degree):
    """Replicates bs from splines package in R."""
    padded_knots = np.hstack([[x.min()] * (degree + 1), knots, [x.max()] * (degree + 1)])
    return pd.DataFrame(
        BSpline(padded_knots, np.eye(len(padded_knots) - degree - 1), degree)(x)[:, 1:],
        index=x.index,
    )


knots = np.quantile(wells.loc[train_id, "logarsenic"], np.linspace(0.1, 0.9, num=10))
B1 = bs(wells["logarsenic"], knots=knots, degree=3)
knots = np.quantile(wells.loc[train_id, "logarsenic"], np.linspace(0.05, 0.95, num=10))
B2 = bs(wells["logarsenic"], knots=knots, degree=3)
knots = np.quantile(wells.loc[train_id, "dist100"], np.linspace(0.1, 0.9, num=10))
B3 = bs(wells["dist100"], knots=knots, degree=3)

In [5]:
wells["intercept"] = 1
X = wells.loc[
    train_id,
    ["intercept", "dist100", "arsenic", "assoc", "edu1", "edu2", "edu3"],
].to_numpy()
X2 = wells.loc[
    train_id,
    ["intercept", "dist100", "logarsenic", "assoc", "edu1", "edu2", "edu3"],
].to_numpy()
X3 = wells.loc[
    train_id,
    [
        "intercept",
        "dist100",
        "arsenic",
        "asthird",
        "asSquare",
        "assoc",
        "edu1",
        "edu2",
        "edu3",
    ],
].to_numpy()
X4 = pd.concat(
    [
        wells.loc[
            train_id,
            ["intercept", "dist100", "assoc", "edu1", "edu2", "edu3"],
        ],
        B1.loc[train_id],
    ],
    axis=1,
).to_numpy()
X5 = pd.concat(
    [
        wells.loc[
            train_id,
            ["intercept", "logarsenic", "assoc", "edu1", "edu2", "edu3"],
        ],
        B3.loc[train_id],
    ],
    axis=1,
).to_numpy()
X6 = wells.loc[train_id, ["intercept", "dist100", "logarsenic", "assoc", "educ"]].to_numpy()

X_test = wells.loc[
    test_id,
    ["intercept", "dist100", "arsenic", "assoc", "edu1", "edu2", "edu3"],
].to_numpy()
X2_test = wells.loc[
    test_id,
    ["intercept", "dist100", "logarsenic", "assoc", "edu1", "edu2", "edu3"],
].to_numpy()
X3_test = wells.loc[
    test_id,
    [
        "intercept",
        "dist100",
        "arsenic",
        "asthird",
        "asSquare",
        "assoc",
        "edu1",
        "edu2",
        "edu3",
    ],
].to_numpy()
X4_test = pd.concat(
    [
        wells.loc[
            test_id,
            ["intercept", "dist100", "assoc", "edu1", "edu2", "edu3"],
        ],
        B1.loc[test_id],
    ],
    axis=1,
).to_numpy()
X5_test = pd.concat(
    [
        wells.loc[
            test_id,
            ["intercept", "logarsenic", "assoc", "edu1", "edu2", "edu3"],
        ],
        B3.loc[test_id],
    ],
    axis=1,
).to_numpy()
X6_test = wells.loc[test_id, ["intercept", "dist100", "logarsenic", "assoc", "educ"]].to_numpy()

In [6]:
train_x_list = [X, X2, X3, X4, X5, X6]
train_x_test_list = [X_test, X2_test, X3_test, X4_test, X5_test, X6_test]
K = len(train_x_list)

## 3. Train six different models

In [9]:
def logistic(X_train, y_train):
    coords = {"features": np.arange(X_train.shape[1])}
    with pm.Model(check_bounds=False, coords=coords) as model:
        beta = pm.Normal(
            "beta",
            mu=0,
            sigma=3,
            dims="features",
        )
        X = pm.Data("X", X_train)

        probs = pm.Deterministic("probs", pm.math.invlogit(pm.math.matrix_dot(X_train, beta)))

        pm.Bernoulli("obs", p=probs, observed=y_train)

    return model

In [10]:
fit_list = []
for k in range(K):
    model = logistic(train_x_list[k], y_train)
    with model:
        idata = pm.sample(draws=1000, tune=1000, return_inferencedata=True)
    fit_list.append(idata)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 6 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 6 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 12 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 74 seconds.
The number of effective samples is smaller than 25% for some parameters.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 88 seconds.
The number of effective samples is smaller than 25% for some parameters.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 7 seconds.


## 4. Get estimates of leave-one-out cross-validated score for each training point

In [14]:
def find_point_wise_loo_score(fit):
    return az.loo(fit, pointwise=True, scale="log").loo_i.values


lpd_point = np.vstack([find_point_wise_loo_score(fit) for fit in fit_list]).T

lpd_point.shape

/home/marco/miniconda3/envs/pymc3-dev-py38/lib/python3.8/site-packages/arviz/stats/stats.py:655: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(


(2000, 6)

In [15]:
dist100_median = wells.loc[wells.index[train_id], "dist100"].median()
logarsenic_median = wells.loc[wells.index[train_id], "logarsenic"].median()
wells["dist100_l"] = (wells["dist100"] - dist100_median).clip(upper=0)
wells["dist100_r"] = (wells["dist100"] - dist100_median).clip(lower=0)
wells["logarsenic_l"] = (wells["logarsenic"] - logarsenic_median).clip(upper=0)
wells["logarsenic_r"] = (wells["logarsenic"] - logarsenic_median).clip(lower=0)

X_stacking_train = wells.loc[
    train_id,
    [
        "edu0",
        "edu1",
        "edu2",
        "edu3",
        "assoc_half",
        "dist100_l",
        "dist100_r",
        "logarsenic_l",
        "logarsenic_r",
    ],
].to_numpy()
X_stacking_test = wells.loc[
    test_id,
    [
        "edu0",
        "edu1",
        "edu2",
        "edu3",
        "assoc_half",
        "dist100_l",
        "dist100_r",
        "logarsenic_l",
        "logarsenic_r",
    ],
].to_numpy()

## 6. Define stacking model

In [16]:
def stacking(
    X,
    N,
    d,
    d_discrete,
    X_test,
    N_test,
    lpd_point,
    K,
    tau_mu,
    tau_sigma,
):
    coords = {
        "continuous_features": np.arange(d - d_discrete),
        "discrete_features": np.arange(d_discrete),
        "features": np.arange(d),
        "models": np.arange(K),
        "models_minus_1": np.arange(K - 1),
    }
    with pm.Model(check_bounds=False, coords=coords) as model:
        exp_lpd_point = pm.math.exp(lpd_point)

        beta_con = pm.Normal(
            "beta_con", mu=0, sigma=1, dims=("models_minus_1", "continuous_features")
        )
        tau = pm.Normal(
            "tau",
            mu=0,
            sigma=1,
            dims=("models_minus_1", "discrete_features"),
        )

        mu = pm.Normal(
            "mu",
            mu=0,
            sigma=tau_mu,
            dims="models_minus_1",
        )
        sigma = pm.HalfNormal(
            "sigma",
            sigma=tau_sigma,
            dims="models_minus_1",
        )

        beta = pm.Deterministic(
            "beta",
            pm.math.stack(
                [
                    pm.math.concatenate([mu[k] + sigma[k] * tau[k], beta_con[k]])
                    for k in range(K - 1)
                ],
            ),
        )

        f = pm.Deterministic(
            "f",
            pm.math.concatenate(
                [
                    pm.math.stack(
                        [pm.math.matrix_dot(X, beta[k]) for k in range(K - 1)],
                        axis=1,
                    ),
                    aesara.tensor.zeros((N, 1)),
                ],
                axis=1,
            ),
        )
        w = pm.Deterministic("w", softmax(f))

        logp = pm.math.log((exp_lpd_point * w).sum(axis=1))

        pm.Potential("logp", pm.math.sum(logp))

        f_test = pm.Deterministic(
            "f_test",
            pm.math.concatenate(
                [
                    pm.math.stack(
                        [pm.math.matrix_dot(X_test, beta[k]) for k in range(K - 1)],
                        axis=1,
                    ),
                    aesara.tensor.zeros((N_test, 1)),
                ],
                axis=1,
            ),
        )

    return model

In [17]:
model = stacking(
    X=X_stacking_train,
    N=X_stacking_train.shape[0],
    d=X_stacking_train.shape[1],
    d_discrete=4,
    X_test=X_stacking_test,
    N_test=X_stacking_test.shape[0],
    lpd_point=lpd_point,
    K=lpd_point.shape[1],
    tau_mu=1.0,
    tau_sigma=0.5,
)
with model:
    idata = pm.sample(draws=1000, tune=1000, return_inferencedata=True, target_accept=0.9)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_con, tau, mu, sigma]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 18 seconds.


In [18]:
w_test = scipy.special.softmax(idata.posterior.f_test.stack(sample=("chain", "draw")), axis=1).mean(
    dim="sample"
)

## 8. Make predictions

In [19]:
train_preds = []
for k in range(K):
    train_pred = scipy.special.expit(
        np.matmul(
            train_x_list[k], fit_list[k].posterior.beta.stack(samples=("chain", "draw")).values
        )
    )
    train_preds.append(train_pred.mean(axis=1))
train_preds = np.vstack(train_preds).T

In [20]:
preds = []
for k in range(K):
    post_pred = scipy.special.expit(
        np.matmul(
            train_x_test_list[k], fit_list[k].posterior.beta.stack(samples=("chain", "draw")).values
        )
    )
    preds.append(post_pred.mean(axis=1))
preds = np.vstack(preds).T

In [21]:
pooled_predictions = (w_test * preds).sum(axis=1)

In [22]:
pooled_predictions.shape

(1020,)

## 9. Predict on test set

## use hierarchical stacking

In [23]:
stats.bernoulli(p=pooled_predictions).logpmf(y_test).mean()

-0.6481014338818963

## using model with best lpd (model selection)

In [24]:
stats.bernoulli(p=preds[:, lpd_point.sum(axis=0).argmax()]).logpmf(y_test).mean()

-0.6514381304301404

## weight according to LOO score

In [25]:
stats.bernoulli(p=(scipy.special.softmax(lpd_point.sum(axis=0)) * preds).sum(axis=1)).logpmf(
    y_test
).mean()

-0.650505494921021